In [1]:
import time, os, random, threading
import pandas as pd
from sklearn import preprocessing

#### Last update on April, 4th ########
# This script just process the dataset and saves a csv file with the correct format.

##TODO: adequate functions to Cloudbook

fulldataset = "./ugredux1g.csv"
#fulldataset = "C:/Users/albercam/Desktop/july.week5.csv.uniqblacklistremoved"

outpath = "./output"
#outpath = "C:/Users/albercam/Desktop/output"

ENCODING = 'utf-8'

done = []
assigned = []

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable. The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

# DU_0 DOES THIS
def split_orig_file():
    print("########################### SPLITTING ORIG #################################")
    actual = time.time()
    print("Actual time", actual)
    lines_per_file = 5000000
    smallfile = None
    with open(fulldataset) as bigfile:
        for lineno, line in enumerate(bigfile):
            if lineno % lines_per_file == 0:
                if smallfile:
                    smallfile.close()
                    print("File done", lineno)
                    done.append('file_{}.csv'.format(lineno))
                small_filename = 'file_{}.csv'.format(lineno + lines_per_file)
                smallfile = open(outpath+"/"+small_filename, "w")
            smallfile.write(line)
        if smallfile:
            smallfile.close()
            print("File done")

    print("It took", time.time()-actual)
    print("########################### DONE SPLITTING #################################")


###############################################################
# HERE WE MUST ASSIGN WHAT DU EXECUTES WHAT FILES             #
###############################################################
def assign_pieces():
    time.sleep(60)
    print(done)
    print(len(done))
    while(len(done)==0):
        time.sleep(5)
    chosen = random.choice(done)
    while(chosen in assigned):
        chosen = random.choice(done)
    assigned.append(chosen)
    print("The chosen is", chosen)
    return chosen



def process_dataset_file():
    #For assigned file execute...
    print("########################### PROCESSING ASSIGNED PIECES #################################")
    count = 0
    for filename in os.listdir(outpath):
        if filename.endswith(".csv"):
            print("Processing", filename)
            actual = time.time()
            #print("Actual time", actual)
            ## Reading the file using chunksize to use less memory. Afterwards it concatenate all the columns into an unique dataframe.
            df1 = pd.read_csv(outpath+"/"+filename,encoding=ENCODING,low_memory=False,error_bad_lines=False,header=None,chunksize=100000)
            df = pd.concat(df1, ignore_index=True)

            df.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

            #Set columns name and attach it to the dataframe.
            colname = ["Timestamp","Duration","Src_IP","Dst_IP","Src_Port","Dest_Port","Proto","Flags","Forward_Status","Service_type","Number_of_Packets","Bytes","Result"]
            df.columns=colname

            #Lets normalize some of the values of the dataset
            encode_numeric_zscore(df, 'Duration')
            encode_text_index(df, 'Src_IP')
            encode_text_index(df, 'Dst_IP')
            encode_text_index(df, 'Src_Port')
            encode_text_index(df, 'Dest_Port')
            encode_text_dummy(df, 'Proto')
            encode_text_dummy(df, 'Flags')
            encode_numeric_zscore(df, 'Forward_Status')
            encode_numeric_zscore(df, 'Number_of_Packets')
            encode_numeric_zscore(df, 'Bytes')

            #Drop timestamps -> for now, they are not useful.
            df.drop(columns=['Timestamp'],axis=1,inplace=True)

            #If background set to 0, if attack set to 1
            df.loc[df.Result != 'background', 'Result'] = 1
            df.loc[df.Result == 'background', 'Result'] = 0
            count+=1
            df.to_csv(outpath+'/out'+str(count)+'.csv', header=False, index=False)
            print("Dataset piece processed in", time.time()-actual)
        
        print("########################### PIECES PROCESSED #################################")


#DU_0 DOES THIS
#Finally everything must be joined:
def create_final_dataset():
    print("########################### PROCESSING FINAL DATASET #################################")
    with open(outpath+"/DATASET.csv", 'w') as outfile:
        for fname in os.listdir(outpath):
            if fname.find("out")!= -1:
                print(fname)
                with open(outpath+"/"+fname) as infile:
                    for line in infile:
                        outfile.write(line)
    print("################################## DATASET READY #####################################")



#split_orig_file()
#assign_pieces()

threading.Thread(target=split_orig_file, args=()).start()

worker1 = threading.Thread(target=assign_pieces, args=())
worker1.start()
worker2 = threading.Thread(target=assign_pieces, args=())
worker2.start()
worker3 = threading.Thread(target=assign_pieces, args=())
worker3.start()


########################### SPLITTING ORIG #################################
Actual time 1572336887.4413905
File done
It took 3.4997568130493164
########################### DONE SPLITTING #################################
[][][]
0

0

0
